## Working with Polars and Scikit-learn

In [1]:
import polars as pl
import numpy as np

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.ensemble import HistGradientBoostingClassifier,HistGradientBoostingRegressor

from sklearn.metrics import root_mean_squared_error,accuracy_score

In [2]:
csv_file = "data/titanic.csv"

In [3]:
df = pl.read_csv(csv_file).select(
    "Pclass","Sex","Age","Fare","Embarked","Survived"
    )

df.head()

Pclass,Sex,Age,Fare,Embarked,Survived
i64,str,f64,f64,str,i64
3,"""male""",22.0,7.25,"""S""",0
1,"""female""",38.0,71.2833,"""C""",1
3,"""female""",26.0,7.925,"""S""",1
1,"""female""",35.0,53.1,"""S""",1
3,"""male""",35.0,8.05,"""S""",0


## Fitting a model with Polars

We can pass a Polars `DataFrame` and `Series` directly to Scikit-learn.

In [4]:
model = LogisticRegression()

x = df.select(
    pl.col("Age", "Fare").fill_null(strategy="mean")
)

y = df["Survived"]

Fit the model

In [5]:
model.fit(x, y)

,"penalty penalty: {'l1', 'l2', 'elasticnet', None}, default='l2'Specify the norm of the penalty:- `None`: no penalty is added;- `'l2'`: add a L2 penalty term and it is the default choice;- `'l1'`: add a L1 penalty term;- `'elasticnet'`: both L1 and L2 penalty terms are added... warning:: Some penalties may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionadded:: 0.19 l1 penalty with SAGA solver (allowing 'multinomial' + L1).. deprecated:: 1.8 `penalty` was deprecated in version 1.8 and will be removed in 1.10. Use `l1_ratio` instead. `l1_ratio=0` for `penalty='l2'`, `l1_ratio=1` for `penalty='l1'` and `l1_ratio` set to any float between 0 and 1 for `'penalty='elasticnet'`.",'deprecated'
,"C C: float, default=1.0Inverse of regularization strength; must be a positive float.Like in support vector machines, smaller values specify strongerregularization. `C=np.inf` results in unpenalized logistic regression.For a visual example on the effect of tuning the `C` parameterwith an L1 penalty, see::ref:`sphx_glr_auto_examples_linear_model_plot_logistic_path.py`.",1.0
,"l1_ratio l1_ratio: float, default=0.0The Elastic-Net mixing parameter, with `0 <= l1_ratio <= 1`. Setting`l1_ratio=1` gives a pure L1-penalty, setting `l1_ratio=0` a pure L2-penalty.Any value between 0 and 1 gives an Elastic-Net penalty of the form`l1_ratio * L1 + (1 - l1_ratio) * L2`... warning:: Certain values of `l1_ratio`, i.e. some penalties, may not work with some solvers. See the parameter `solver` below, to know the compatibility between the penalty and solver... versionchanged:: 1.8 Default value changed from None to 0.0... deprecated:: 1.8 `None` is deprecated and will be removed in version 1.10. Always use `l1_ratio` to specify the penalty type.",0.0
,"dual dual: bool, default=FalseDual (constrained) or primal (regularized, see also:ref:`this equation `) formulation. Dual formulationis only implemented for l2 penalty with liblinear solver. Prefer `dual=False`when n_samples > n_features.",False
,"tol tol: float, default=1e-4Tolerance for stopping criteria.",0.0001
,"fit_intercept fit_intercept: bool, default=TrueSpecifies if a constant (a.k.a. bias or intercept) should beadded to the decision function.",True
,"intercept_scaling intercept_scaling: float, default=1Useful only when the solver `liblinear` is usedand `self.fit_intercept` is set to `True`. In this case, `x` becomes`[x, self.intercept_scaling]`,i.e. a ""synthetic"" feature with constant value equal to`intercept_scaling` is appended to the instance vector.The intercept becomes``intercept_scaling * synthetic_feature_weight``... note:: The synthetic feature weight is subject to L1 or L2 regularization as all other features. To lessen the effect of regularization on synthetic feature weight (and therefore on the intercept) `intercept_scaling` has to be increased.",1
,"class_weight class_weight: dict or 'balanced', default=NoneWeights associated with classes in the form ``{class_label: weight}``.If not given, all classes are supposed to have weight one.The ""balanced"" mode uses the values of y to automatically adjustweights inversely proportional to class frequencies in the input dataas ``n_samples / (n_classes * np.bincount(y))``.Note that these weights will be multiplied with sample_weight (passedthrough the fit method) if sample_weight is specified... versionadded:: 0.17 *class_weight='balanced'*",None
,"random_state random_state: int, RandomState instance, default=NoneUsed when ``solver`` == 'sag', 'saga' or 'liblinear' to shuffle thedata. See :term:`Glossary ` for details.",None
,"solver solver: {'lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'}, default='lbfgs'Algorithm to use in the optimization problem. Default is 'lbfgs'.To choose a solver, you might want to consider the following aspects:- 'lbfgs' is a good default solver because it works reasonably well for a wide class of problems.- For :term:`mul

In [ ]:
pred_df = pl.DataFrame(
    {
        "label": y, 
        "pred": model.predict(x)
    }
)

print(f"Accuracy: {100*accuracy_score(pred_df["label"],pred_df["pred"]):.2f}%")

Accuracy: 65.66%


## Categorical features in a gradient boosting model

Typically we have to encode categorical features manually. 

However, Scikit-learn's gradient boosting model `HistGradientBoostingClassifier` or `HistGradientBoostingRegressor` allows to pass a Polars `pl.Categorical` column directly to the model without encoding it.

### Split the train and test data

In [7]:
df_train, df_test = train_test_split(
    df.select("Pclass", "Age", "Fare","Sex","Survived"),
    test_size=0.2,
    random_state=0
)

### Separate X and y data

In [8]:
X_train = df_train.select(
    pl.col("Pclass").cast(pl.String).cast(pl.Categorical),
    pl.col("Sex").cast(pl.Categorical),
    pl.col("Age").fill_null(pl.col("Age").median()),
    pl.col("Fare")
)

y_train = df_train["Survived"]

### Fit the gradient boosting model

In [9]:
model_grad_boost = HistGradientBoostingClassifier(categorical_features="from_dtype")

model_grad_boost.fit(X_train, y_train)

,"loss loss: {'log_loss'}, default='log_loss'The loss function to use in the boosting process.For binary classification problems, 'log_loss' is also known as logistic loss,binomial deviance or binary crossentropy. Internally, the model fits one treeper boosting iteration and uses the logistic sigmoid function (expit) asinverse link function to compute the predicted positive class probability.For multiclass classification problems, 'log_loss' is also known as multinomialdeviance or categorical crossentropy. Internally, the model fits one tree perboosting iteration and per class and uses the softmax function as inverse linkfunction to compute the predicted probabilities of the classes.",'log_loss'
,"learning_rate learning_rate: float, default=0.1The learning rate, also known as *shrinkage*. This is used as amultiplicative factor for the leaves values. Use ``1`` for noshrinkage.",0.1
,"max_iter max_iter: int, default=100The maximum number of iterations of the boosting process, i.e. themaximum number of trees for binary classification. For multiclassclassification, `n_classes` trees per iteration are built.",100
,"max_leaf_nodes max_leaf_nodes: int or None, default=31The maximum number of leaves for each tree. Must be strictly greaterthan 1. If None, there is no maximum limit.",31
,"max_depth max_depth: int or None, default=NoneThe maximum depth of each tree. The depth of a tree is the number ofedges to go from the root to the deepest leaf.Depth isn't constrained by default.",None
,"min_samples_leaf min_samples_leaf: int, default=20The minimum number of samples per leaf. For small datasets with lessthan a few hundred samples, it is recommended to lower this valuesince only very shallow trees would be built.",20
,"l2_regularization l2_regularization: float, default=0The L2 regularization parameter penalizing leaves with small hessians.Use ``0`` for no regularization (default).",0.0
,"max_features max_features: float, default=1.0Proportion of randomly chosen features in each and every node split.This is a form of regularization, smaller values make the trees weakerlearners and might prevent overfitting.If interaction constraints from `interaction_cst` are present, only allowedfeatures are taken into account for the subsampling... versionadded:: 1.4",1.0
,"max_bins max_bins: int, default=255The maximum number of bins to use for non-missing values. Beforetraining, each feature of the input array `X` is binned intointeger-valued bins, which allows for a much faster training stage.Features with a small number of unique values may use less than``max_bins`` bins. In addition to the ``max_bins`` bins, one more binis always reserved for missing values. Must be no larger than 255.",255
,"categorical_features categorical_features: array-like of {bool, int, str} of shape (n_features) or shape (n_categorical_features,), default='from_dtype'Indicates the categorical features.- None : no feature will be considered categorical.- boolean array-like : boolean mask indicating categorical features.- integer array-like : integer indices indicating categorical features.- str array-like: names of categorical features (assuming the training data has feature names).- `""from_dtype""`: dataframe columns with dtype ""category"" are considered to be categorical features. The input must be an object exposing a ``__dataframe__`` method such as pandas or polars DataFrames to use this feature.For each categorical feature, there must be at most `max_bins` uniquecategories. Negative values for categorical features encoded as numericdtypes are treated as missing values. All categorical values areconverted to floating point numbers. This means that categorical valuesof 1.0 and 1 are treated as the same category.Read more in the :ref:`User Guide `... versionadded:: 0.24.. versionchanged:: 1.2 Added support for feature names... versionchanged:: 1.4 Added `""from_dtype""` option... versionchanged:: 1.6 The default value changed from `None` to `""from_dtype""`.",'from_dt

### Make test data

In [10]:
X_test = df_test.with_columns(
        pl.lit(df_train["Age"].median()).alias("Age_median")
    ).select(
        pl.col("Pclass").cast(pl.String).cast(pl.Categorical),
        pl.col("Sex").cast(pl.Categorical),
        pl.col("Age").fill_null(pl.col("Age_median")),
        pl.col("Fare")
    )

y_test = df_test["Survived"]

### Predict

In [11]:
pred_df = pl.DataFrame(
    {
        "label":y_test,
        "pred":model_grad_boost.predict(X_test)
    }
)

print(f"Accuracy: {100*accuracy_score(pred_df["label"],pred_df["pred"]):.2f}%")

Accuracy: 85.47%


## Scikit-learn pipelines

A more systematic way to produce ML pipelines is to use Scikit-learn's preprocessing tools: 
- `Pipeline` to compose multiple transformation steps on a column together
- `ColumnTransformer` to run transformations on multiple columns together
- `Pipeline` to compose the pre-processing and model fit/predict steps together

In [13]:
df_train, df_test = train_test_split(
    df.select("Pclass","Age","Fare","Sex","Survived"),
    test_size=0.2, 
    random_state=0
)

X_train,y_train = df_train.drop("Survived"),df_train["Survived"]
X_test,y_test = df_test.drop("Survived"),df_test["Survived"]

### First pipeline for numerical features

In [14]:
numeric_features = ["Age", "Fare"]

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler())
    ]
)

### Second pipeline for categorical features

Set `sparse_output` to False to keep the output is Polars `DataFrame`

In [15]:
categorical_features = ["Sex", "Pclass"]

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ]
)

### ColumnTransformer

In [16]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

### Final pipeline

In [18]:
preprocess_model_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("classifier", LogisticRegression())
    ]
)

preprocess_model_pipeline.set_output(transform="polars")

preprocess_model_pipeline.fit(X_train, y_train)

print("model score: %.3f" % preprocess_model_pipeline.score(X_test, y_test))

pred_df = pl.DataFrame(
    {
        "label": y_test,
        "pred": preprocess_model_pipeline.predict(X_test)
    }
)

pred_df

model score: 0.799


label,pred
i64,i64
0,0
0,0
0,0
1,1
1,1
…,…
1,1
0,0
1,1


## Exercises

### Exercise one
We want to predict the tip amount in the NYC taxi database from the other columns

In [20]:
taxi_data = "data/nyc_trip_data_1k.csv"
df_nyc = pl.read_csv(taxi_data,try_parse_dates=True)
df_nyc

VendorID,pickup,dropoff,passenger_count,trip_distance,fare_amount,tip_amount
str,datetime[μs],datetime[μs],f64,f64,f64,f64
"""id1""",2022-01-01 00:04:14,2022-01-01 00:26:12,1.0,10.83,31.0,0.0
"""id2""",2022-01-01 00:32:17,2022-01-01 00:49:23,1.0,3.97,14.5,3.66
"""id8""",2022-01-01 00:40:58,2022-01-01 01:00:59,4.0,8.44,25.5,0.0
"""id0""",2022-01-01 00:55:13,2022-01-01 01:25:49,1.0,12.61,37.5,12.39
"""id1""",2022-01-01 00:55:24,2022-01-01 01:00:45,1.0,1.49,6.5,0.0
…,…,…,…,…,…,…
"""id0""",2022-01-14 18:20:51,2022-01-14 18:34:09,1.0,1.07,9.0,0.0
"""id4""",2022-01-14 18:32:26,2022-01-14 18:50:36,2.0,5.57,18.0,5.58
"""id2""",2022-01-14 18:34:11,2022-01-14 18:39:18,3.0,0.92,5.5,2.45


Randomly split `df_nyc` into a train and test `DataFrames`

In [21]:
df_train, df_test = train_test_split(
    df_nyc,
    test_size=0.2,
    random_state=0
)

Create `X_train` with:
- `VendorID` as a categorical variable
- `trip_minutes` as the duration of the trip in minutes
- `passenger_count`,`trip_distance`,`fare_amount` as numerical features

and `y_train` with `tip_amount` as the target vector

In [22]:
X_train = df_train.with_columns(
    pl.col("VendorID").cast(pl.Categorical),
    trip_minutes = (pl.col("dropoff") - pl.col("pickup")).dt.total_minutes()
).drop("pickup","dropoff","tip_amount")

y_train = df_train["tip_amount"]

Instantiate and fit the gradient boosting model

In [23]:
model = HistGradientBoostingRegressor(categorical_features="from_dtype")

model.fit(X_train, y_train)

,"loss loss: {'squared_error', 'absolute_error', 'gamma', 'poisson', 'quantile'}, default='squared_error'The loss function to use in the boosting process. Note that the""squared error"", ""gamma"" and ""poisson"" losses actually implement""half least squares loss"", ""half gamma deviance"" and ""half poissondeviance"" to simplify the computation of the gradient. Furthermore,""gamma"" and ""poisson"" losses internally use a log-link, ""gamma""requires ``y > 0`` and ""poisson"" requires ``y >= 0``.""quantile"" uses the pinball loss... versionchanged:: 0.23 Added option 'poisson'... versionchanged:: 1.1 Added option 'quantile'... versionchanged:: 1.3 Added option 'gamma'.",'squared_error'
,"quantile quantile: float, default=NoneIf loss is ""quantile"", this parameter specifies which quantile to be estimatedand must be between 0 and 1.",None
,"learning_rate learning_rate: float, default=0.1The learning rate, also known as *shrinkage*. This is used as amultiplicative factor for the leaves values. Use ``1`` for noshrinkage.",0.1
,"max_iter max_iter: int, default=100The maximum number of iterations of the boosting process, i.e. themaximum number of trees.",100
,"max_leaf_nodes max_leaf_nodes: int or None, default=31The maximum number of leaves for each tree. Must be strictly greaterthan 1. If None, there is no maximum limit.",31
,"max_depth max_depth: int or None, default=NoneThe maximum depth of each tree. The depth of a tree is the number ofedges to go from the root to the deepest leaf.Depth isn't constrained by default.",None
,"min_samples_leaf min_samples_leaf: int, default=20The minimum number of samples per leaf. For small datasets with lessthan a few hundred samples, it is recommended to lower this valuesince only very shallow trees would be built.",20
,"l2_regularization l2_regularization: float, default=0The L2 regularization parameter penalizing leaves with small hessians.Use ``0`` for no regularization (default).",0.0
,"max_features max_features: float, default=1.0Proportion of randomly chosen features in each and every node split.This is a form of regularization, smaller values make the trees weakerlearners and might prevent overfitting.If interaction constraints from `interaction_cst` are present, only allowedfeatures are taken into account for the subsampling... versionadded:: 1.4",1.0
,"max_bins max_bins: int, default=255The maximum number of bins to use for non-missing values. Beforetraining, each feature of the input array `X` is binned intointeger-valued bins, which allows for a much faster training stage.Features with a small number of unique values may use less than``max_bins`` bins. In addition to the ``max_bins`` bins, one more binis always reserved for missing values. Must be no larger than 255.",255
,"categorical_features categorical_features: array-like of {bool, int, str} of shape (n_features) or shape (n_categorical_features,), default='from_dtype'Indicates the categorical features.- None : no feature will be considered categorical.- boolean array-like : boolean mask indicating categorical features.- integer array-like : integer indices indicating categorical features.- str array-like: names of categorical features (assuming the training data has feature names).- `""from_dtype""`: dataframe columns with dtype ""category"" are considered to be categorical features. The input must be an object exposing a ``__dataframe__`` method such as pandas or polars DataFrames to use this feature.For each categorical feature, there must be at most `max_bins` uniquecategories. Negative values for categorical features encoded as numericdtypes are treated as missing values. All categorical values areconverted to floating point numbers. This means that categorical valuesof 1.0 and 1 are treated as the same category.Read more in the :ref:`User Guide ` and:ref:`sphx_glr_auto_examples_ensemble_plot_gradient_boosting_categorical.py`... versionadded:: 0.24.. versionchanged:: 1.2 Added support for feature names... versionchanged:: 

Make a `DataFrame` with `actual` and `pred` columns on the training data

In [24]:
train_pred_df = pl.DataFrame(
    {
        "actual": y_train,
        "pred": model.predict(X_train)
    }
)

Make a scatter plot of the `actual` versus `pred`

In [31]:
train_pred_df.plot.scatter(
    x="actual",
    y="pred"
)

alt.Chart(...)

Get the root-mean-squared error of the prediction of the tip amount

In [ ]:
rmse = root_mean_squared_error(
    train_pred_df["actual"],
    train_pred_df["pred"]
)

print(f"RMSE: {rmse}")

RMSE: 1.7323741349256456


Create the test feature matrix and target vector

In [28]:
X_test = df_test.with_columns(
    pl.col("VendorID").cast(pl.Categorical),
    trip_minutes = (pl.col("dropoff") - pl.col("pickup")).dt.total_minutes(),
).drop("pickup","dropoff","tip_amount")

y_test = df_test["tip_amount"]

Make predictions on the test `DataFrame` and make a scatterplot

In [34]:
test_pred_df = pl.DataFrame(
    {
        "label":y_test,
        "pred":model.predict(X_test)
    }
)

test_pred_df.plot.scatter(
    x="pred",
    y="label",
)

alt.Chart(...)

Get the root-mean-squared error on the test data

In [37]:
rmse = root_mean_squared_error(
    test_pred_df["label"],
    test_pred_df["pred"]
)

print(f"RMSE: {rmse}")

RMSE: 2.6262628913585466


### Exercise two
We now fit models and make predictions with a `Pipeline` approach.

Begin by creating `df_nyc_train` and `df_nyc_test` by:
- creating any feature columns and
- splitting `df_nyc`

In [38]:
df_nyc_train, df_nyc_test = train_test_split(
    df_nyc.select(
        "VendorID",
        (pl.col("dropoff") - pl.col("pickup")).dt.total_minutes().alias("trip_minutes"),
        "passenger_count",
        "trip_distance",
        "fare_amount",
        "tip_amount"
    ),
    test_size=0.2, 
    random_state=0
)

Now create the train/test feature matrixes and target vectors

In [39]:
X_train, y_train = df_nyc_train.drop("tip_amount"), df_nyc_train["tip_amount"]
X_test,y_test = df_nyc_test.drop("tip_amount"),df_nyc_test["tip_amount"]

Create a pipeline for the numerical features by:
- imputing missing values with the median
- scaling values by the min/max of that column

In [40]:
from sklearn.preprocessing import MinMaxScaler

numeric_features = ["trip_minutes", "passenger_count","trip_distance","fare_amount",]

numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", MinMaxScaler())
    ]
)

Create the categorical `VendorID` feature

In [41]:
categorical_features = ["VendorID"]

categorical_transformer = Pipeline(
    steps=[
        ("encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
    ]
)

Make a `ColumnTransformer` to preprocess all features

In [42]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features)
    ]
)

Make a `Pipeline` to preprocess the features and do **linear regression**

In [43]:
preprocess_model_pipeline = Pipeline(
    steps=[
        ("preprocessor", preprocessor),
        ("model", LinearRegression())
    ]
)

preprocess_model_pipeline.set_output(transform="polars")

preprocess_model_pipeline.fit(X_train, y_train)

,"steps steps: list of tuplesList of (name of step, estimator) tuples that are to be chained insequential order. To be compatible with the scikit-learn API, all stepsmust define `fit`. All non-last steps must also define `transform`. See:ref:`Combining Estimators ` for more details.","[('preprocessor', ...), ('model', ...)]"
,"transform_input transform_input: list of str, default=NoneThe names of the :term:`metadata` parameters that should be transformed by thepipeline before passing it to the step consuming it.This enables transforming some input arguments to ``fit`` (other than ``X``)to be transformed by the steps of the pipeline up to the step which requiresthem. Requirement is defined via :ref:`metadata routing `.For instance, this can be used to pass a validation set through the pipeline.You can only set this if metadata routing is enabled, which youcan enable using ``sklearn.set_config(enable_metadata_routing=True)``... versionadded:: 1.6",None
,"memory memory: str or object with the joblib.Memory interface, default=NoneUsed to cache the fitted transformers of the pipeline. The last stepwill never be cached, even if it is a transformer. By default, nocaching is performed. If a string is given, it is the path to thecaching directory. Enabling caching triggers a clone of the transformersbefore fitting. Therefore, the transformer instance given to thepipeline cannot be inspected directly. Use the attribute ``named_steps``or ``steps`` to inspect estimators within the pipeline. Caching thetransformers is advantageous when fitting is time consuming. See:ref:`sphx_glr_auto_examples_neighbors_plot_caching_nearest_neighbors.py`for an example on how to enable caching.",None
,"verbose verbose: bool, default=FalseIf True, the time elapsed while fitting each step will be printed as itis completed.",False
,"transformers transformers: list of tuplesList of (name, transformer, columns) tuples specifying thetransformer objects to be applied to subsets of the data.name : str Like in Pipeline and FeatureUnion, this allows the transformer and its parameters to be set using ``set_params`` and searched in grid search.transformer : {'drop', 'passthrough'} or estimator Estimator must support :term:`fit` and :term:`transform`. Special-cased strings 'drop' and 'passthrough' are accepted as well, to indicate to drop the columns or to pass them through untransformed, respectively.columns : str, array-like of str, int, array-like of int, array-like of bool, slice or callable Indexes the data on its second axis. Integers are interpreted as positional columns, while strings can reference DataFrame columns by name. A scalar string or int should be used where ``transformer`` expects X to be a 1d array-like (vector), otherwise a 2d array will be passed to the transformer. A callable is passed the input data `X` and can return any of the above. To select multiple columns by name or dtype, you can use :obj:`make_column_selector`.","[('num', ...), ('cat', ...)]"
,"remainder remainder: {'drop', 'passthrough'} or estimator, default='drop'By default, only the specified columns in `transformers` aretransformed and combined in the output, and the non-specifiedcolumns are dropped. (default of ``'drop'``).By specifying ``remainder='passthrough'``, all remaining columns thatwere not specified in `transformers`, but present in the data passedto `fit` will be automatically passed through. This subset of columnsis concatenated with the output of the transformers. For dataframes,extra columns not seen during `fit` will be excluded from the outputof `transform`.By setting ``remainder`` to be an estimator, the remainingnon-specified columns will use the ``remainder`` estimator. Theestimator must support :term:`fit` and :term:`transform`.Note that using this feature requires that the DataFrame columnsinput at :term:`fit` and :term:`transform` have identical order.",'drop'
,"sparse_threshold sparse_threshold: float, default=0.3If the output of the different transformers co

Make a `DataFrame` of actual and predicted values on the **test** data

In [44]:
pred_df = pl.DataFrame(
    {
        "actual": y_test,
        "pred": preprocess_model_pipeline.predict(X_test)
    }
)
pred_df

actual,pred
f64,f64
1.0,2.044082
0.0,2.432771
3.56,1.68788
2.36,1.032443
2.66,1.221535
…,…
12.62,7.543725
0.0,1.340686
2.76,2.186759


Get the RMSE of the model on the test data

In [45]:
rmse = root_mean_squared_error(pred_df["actual"], pred_df["pred"])

print(f"RMSE: {rmse}")

RMSE: 2.3065721997757054
